### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [30]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),NaN
2,0000012,The Arrival of a Train (1896),NaN
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [31]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0111161,10,1373234211
1,1,0117060,7,1373415231
2,1,0120755,6,1373424360
3,1,0317919,6,1373495763
4,1,0454876,10,1373621125


In [32]:
# more work
movies.describe()

,movie_id,movie,genre
count,33598,33598,25104
unique,33598,33543,2270
top,1096886,Home (2016),Drama
freq,1,4,2494


In [33]:
# and still more
reviews.describe()

,rating
count,786173.000000
mean,7.300929
std,1.854783
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,10.000000


In [34]:
# and even more - open additional cells as necessary
movies.isnull().sum()

movie_id       0
movie          0
genre       8494
dtype: int64

In [35]:
reviews.isnull().sum()

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

In [36]:
genres = []
for genre in movies['genre']:
    try:
        genres.extend(genre.split('|'))
    except:
        pass

genres = set(genres)
print(genres)
print(len(set(genres)))

{'Biography', 'Horror', 'Fantasy', 'History', 'Animation', 'Drama', 'Music', 'Musical', 'Sport', 'Adult', 'Talk-Show', 'News', 'Game-Show', 'Comedy', 'Thriller', 'Reality-TV', 'Action', 'Sci-Fi', 'Romance', 'Documentary', 'Western', 'Film-Noir', 'Crime', 'Adventure', 'Family', 'War', 'Short', 'Mystery'}
28


In [37]:
reviews['user_id'].nunique()

57855

In [38]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset':  33598,
'The number of ratings in the dataset': 786173,
'The number of different genres': 28,
'The number of unique users in the dataset': 57855,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.300929,
'The minimum rating given across all ratings': 0, 
'The maximum rating given across all ratings': 10.000000
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [39]:
# cell for work
# Pull the date out of the title and create a new column
movies['date'] = movies['movie'].str.split(' ').str[-1].str[1:-1]
movies['date'].head()

0    1894
1    1895
2    1896
3    1895
4    1896
Name: date, dtype: object

In [40]:
# and another
def convert_year_to_century(year):
    if str(year).startswith('18'):
        return '1800s'
    elif str(year).startswith('19'):
        return '1900s'
    elif str(year).startswith('20'):
        return '2000s'
    else:
        raise ValueError(f"Can convert value of {year}")
        
movies['date'] = movies['date'].apply(lambda year: convert_year_to_century(year))
movies['date'].head()

0    1800s
1    1800s
2    1800s
3    1800s
4    1800s
Name: date, dtype: object

In [41]:
# Dummy the centuries
movies = pd.get_dummies(movies, columns=['date'])
movies.head()

,movie_id,movie,genre,date_1800s,date_1900s,date_2000s
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1,0,0
1,0000010,La sortie des usines Lumière (1895),NaN,1,0,0
2,0000012,The Arrival of a Train (1896),NaN,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1,0,0


In [42]:
# Drop rows with missing genre
movies = movies.dropna(subset=['genre'])
movies.head()

,movie_id,movie,genre,date_1800s,date_1900s,date_2000s
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1,0,0
6,0000417,Le voyage dans la lune (1902),Short|Action|Adventure|Comedy|Fantasy|Sci-Fi,0,1,0
7,0000439,The Great Train Robbery (1903),Short|Action|Crime|Western,0,1,0
10,0000833,The Country Doctor (1909),Short|Drama,0,1,0


In [43]:
# Dummy the 'genre' column
def split_genre(row_val, genre):
    try:
        if genre in row_val:
            return 1
        else:
            return 0
    except:
        raise
        
for g in genres:
    movies[g] = movies['genre'].apply(lambda row_val: split_genre(row_val, g))

In [44]:
movies.head()

,movie_id,movie,genre,date_1800s,date_1900s,date_2000s,Biography,Horror,Fantasy,History,...,Romance,Documentary,Western,Film-Noir,Crime,Adventure,Family,War,Short,Mystery
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0000091,Le manoir du diable (1896),Short|Horror,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
6,0000417,Le voyage dans la lune (1902),Short|Action|Adventure|Comedy|Fantasy|Sci-Fi,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
7,0000439,The Great Train Robbery (1903),Short|Action|Crime|Western,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
10,0000833,The Country Doctor (1909),Short|Drama,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [45]:
# Create a date oout of timestamp
reviews['date'] = pd.to_datetime(reviews['timestamp'], unit='s')
reviews['date'].head()

0   2013-07-07 21:56:51
1   2013-07-10 00:13:51
2   2013-07-10 02:46:00
3   2013-07-10 22:36:03
4   2013-07-12 09:25:25
Name: date, dtype: datetime64[ns]

In [46]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0111161,10,1373234211,2013-07-07 21:56:51
1,1,0117060,7,1373415231,2013-07-10 00:13:51
2,1,0120755,6,1373424360,2013-07-10 02:46:00
3,1,0317919,6,1373495763,2013-07-10 22:36:03
4,1,0454876,10,1373621125,2013-07-12 09:25:25
